In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_extractor import TableExtractor
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper
from series_extraction.excel_validator import ExcelValidator

from ast_transformation.series_formula_generator_old import SeriesFormulaGenerator
from ast_transformation.formula_generator import FormulaGenerator
from ast_transformation.formula_list_generator import FormulaListGenerator
from ast_transformation.formula_evaluator import FormulaEvaluator

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from pipeline_building.series_dependencies_builder import SeriesDependenciesBuilder
from pipeline_building.dag_sorter import DAGSorter

from excel_builder import ExcelBuilder

from excel_checker import ExcelChecker

from excel_data_extractor import ExcelDataExtractor


In [2]:
data_directory = "/Users/chrislittle/GitHub/speedsheet/excel-2-python/data"

project_name = 'vehicle_data'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")
excel_reduced_clean_filepath = os.path.join(data_directory, "excel_files_reduced_clean", f"{project_name}_reduced_clean.xlsx")
excel_raw_clean_filepath = os.path.join(data_directory, "excel_files_raw_clean", f"{project_name}_raw_clean.xlsx")
excel_reduced_clean_series_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series", f"{project_name}_reduced_clean_series.xlsx")
excel_reduced_clean_series_python_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series_python", f"{project_name}_reduced_clean_series_python.xlsx")
excel_raw_clean_series_python_filepath = os.path.join(data_directory, "excel_files_raw_clean_series_python", f"{project_name}_raw_clean_series_python.xlsx")

In [3]:
def get_evaluated_results_from_formula_ast(series_id, formula_ast, series_values_dict):
    formula_list_generator = FormulaListGenerator(formula_ast, series_values_dict)
    values_length = len(series_values_dict[str(series_id)])
    formula_list = formula_list_generator.generate_formula_list(0, values_length-1)
    formula_strings = [str(formula) for formula in formula_list]


    formula_evaluator = FormulaEvaluator()

    results = [formula_evaluator.evaluate_formula(f"={formula_string}") for formula_string in formula_strings]

    return results

In [4]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)
is_valid = ExcelValidator.validate_excel(excel_reduced)
if not is_valid:
    raise Exception("Excel file is not valid")

excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)
excel_raw_clean = ExcelCleaner.clean_excel(excel_raw)
ExcelBuilder.create_excel_from_workbook(excel_reduced_clean.workbook_with_formulas, excel_reduced_clean_filepath)
ExcelBuilder.create_excel_from_workbook(excel_raw_clean.workbook_with_formulas,  excel_raw_clean_filepath)

extracted_tables, workbook_data = TableExtractor.extract_tables(excel_reduced_clean)
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)
if not is_compatible:
    raise Exception("Excel file is not compatible")

series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, workbook_data=workbook_data)
series_mapping = SeriesMapper.map_series(series_dict)
series_iterator = SeriesIterator.iterate_series(series_dict)

series_list = [series for series in series_iterator]

series_list_with_formulas = [
    series for series in series_list if series.formulas != [None, None]
]
series_list_with_values = [
    series for series in series_list if series.formulas == [None, None]
]

generic_formula_dictionary = {}

for series in series_list_with_formulas:
    formula_1, formula_2 = SeriesFormulaGenerator.adjust_formulas(
        series.formulas
    )
    if formula_1 is not None and formula_2 is not None:

        series_implementer = SeriesImplementer(
            series_mapping, sheet_name=series.worksheet.sheet_name
        )

        formula_1_ast = FormulaParser.parse_formula(formula_1)
        formula_1_ast_series = series_implementer.update_ast(formula_1_ast)

        formula_2_ast = FormulaParser.parse_formula(formula_2)
        formula_2_ast_series = series_implementer.update_ast(formula_2_ast)

        generic_formula_ast = FormulaGenerator.traverse_and_replace(formula_1_ast_series, formula_2_ast_series)

        generic_formula_dictionary[series.series_id] = generic_formula_ast



In [5]:
series_values_dict = {str(series.series_id): series.values for series in series_list}

In [6]:
series_values_dict_raw = ExcelDataExtractor.extract_series_data_from_excel(excel_raw.workbook_with_values, series_list)

In [7]:
series_dependencies = SeriesDependenciesBuilder.build_dependencies(generic_formula_dictionary)

In [8]:
sorted_dag = DAGSorter.sort_dag(series_dependencies)

In [9]:
import copy

def create_series_list_updated_raw(sorted_dag, generic_formula_dictionary, series_dict, series_values_dict_raw, series_list_with_values):

    sorted_dag = copy.deepcopy(sorted_dag)
    generic_formula_dictionary = copy.deepcopy(generic_formula_dictionary)
    series_dict = copy.deepcopy(series_dict)
    series_values_dict_raw = copy.deepcopy(series_values_dict_raw)
    series_list_with_values = copy.deepcopy(series_list_with_values)

    series_list_new_raw = []
    for series_id in sorted_dag:
        formula_ast = generic_formula_dictionary.get(series_id)

        series = SeriesMapper.get_series_from_series_id(series_id, series_dict)
        if formula_ast:    
            values = get_evaluated_results_from_formula_ast(series_id, formula_ast, series_values_dict_raw)
            series.values = values
            series.series_length = len(values)
            series_list_new_raw.append(series)

    series_list_with_values_raw = []

    for series in series_list_with_values:
        series_id = series.series_id
        values = series_values_dict_raw[str(series_id)]
        series.values = values
        series.series_length = len(values)
        series_list_with_values_raw.append(series)

    series_list_updated_raw = series_list_new_raw + series_list_with_values_raw

    return series_list_updated_raw

In [10]:
series_list_updated_raw = create_series_list_updated_raw(sorted_dag, generic_formula_dictionary, series_dict, series_values_dict_raw, series_list_with_values)

In [11]:
series_list_updated_reduced = create_series_list_updated_raw(sorted_dag, generic_formula_dictionary, series_dict, series_values_dict, series_list_with_values)

In [12]:
ExcelBuilder.create_excel_from_series(series_list_updated_reduced, excel_reduced_clean_series_python_filepath, values_only=True)

In [13]:
ExcelBuilder.create_excel_from_series(series_list_updated_raw, excel_raw_clean_series_python_filepath, values_only=True)

In [14]:
ExcelChecker.excels_are_equivalent(excel_reduced_clean_series_python_filepath, excel_reduced_clean_filepath)

Values in sheet 'VehicleData' are different
wb1: Automatic
wb2: None


False

In [24]:
ExcelChecker.excels_are_equivalent(excel_raw_clean_series_python_filepath, excel_raw_clean_filepath)

True